In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../elena/data/training_final_latcorr.csv')

In [3]:
df['kWh'] = df['Area_km2'] / df['Area / Electricity']
df = df.drop(['Longitude', 'Latitude','CO2 (g/kWh)', 'CH4 (g/kWh)', 'Area / Electricity', 'Name', 'field_1',
              'temp_spring_avg', 'temp_summer_avg', 'temp_fall_avg', 'temp_winter_avg', 
              'NDVI_spring_avg', 'NDVI_summer_avg', 'NDVI_fall_avg', 'NDVI_winter_avg', 
              'npp_spring_avg' , 'npp_summer_avg' , 'npp_fall_avg' , 'npp_winter_avg'], axis=1)

In [4]:
df.columns

Index(['CO2 (mg C m¯² d¯¹)', 'CH4 (mg C m-2 d-1)', 'Area_km2', 'Age',
       'Volume_km3', 'Areakm2_div_Volkm3', 'org_c', 'temp_annual_avg',
       'temp_spring_avg_lc', 'temp_summer_avg_lc', 'temp_fall_avg_lc',
       'temp_winter_avg_lc', 'NDVI_annual_avg', 'NDVI_spring_avg_lc',
       'NDVI_summer_avg_lc', 'NDVI_fall_avg_lc', 'NDVI_winter_avg_lc',
       'npp_annual_avg', 'npp_spring_avg_lc', 'npp_summer_avg_lc',
       'npp_fall_avg_lc', 'npp_winter_avg_lc', 'erosion', 'kWh'],
      dtype='object')

In [5]:
ch4 = df.drop(['CO2 (mg C m¯² d¯¹)'], axis=1)
ch4 = ch4[ch4['CH4 (mg C m-2 d-1)'].notna()]

In [6]:
from sklearn.impute import KNNImputer
model_impute = KNNImputer(n_neighbors=int(np.sqrt(ch4.shape[0])))
ch4_imputed = model_impute.fit_transform(ch4)

In [7]:
ch4_imputed = pd.DataFrame(columns=ch4.columns, data=ch4_imputed)

In [8]:
from sklearn.preprocessing import StandardScaler
data = ch4_imputed.drop(['CH4 (mg C m-2 d-1)'], axis=1).copy()
scaler = StandardScaler()
ch4_scaled = pd.DataFrame(scaler.fit_transform(data),columns=ch4.columns[1:])

# SVM

In [9]:
X = ch4_scaled
y = ch4['CH4 (mg C m-2 d-1)']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
from sklearn.svm import SVR

In [13]:
ols = LinearRegression()
ols.fit(X,y)
print(ols.score(X,y))
print(ols.coef_)

0.26833377786315404
[  -3.69600441  -19.57040235   -6.37834404   -1.5067939   -24.09351214
 1061.84584217 -262.94813791 -285.59930869  106.15896628 -573.14644937
   -9.83583363  -90.98053869   53.83557977  -22.23258693   97.66593977
 -170.7842906    84.06468313  101.37474312  -21.75564866   83.48490765
    8.26850134  -28.12839469]


In [14]:
svr = SVR()
svr.fit(X,y)
print(svr.score(X,y))

-0.059554261307776635


In [15]:
y_train_scaled = scaler.fit_transform(np.array(y_train).reshape(-1,1))
y_test_scaled = scaler.fit_transform(np.array(y_test).reshape(-1,1))

In [16]:
linear_rbf = SVR(kernel='linear')
linear_rbf.fit(X_train, y_train_scaled)

C:\Users\kkrle\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR(kernel='linear')

In [17]:
print('train score:', linear_rbf.score(X_train, y_train_scaled))
print('test  score:', linear_rbf.score(X_test, y_test_scaled))

train score: 0.3107937238415981
test  score: 0.30146101253017954
